# Differential privacy section project

This work is part of [Udacity](http://udacity.com) 'Secure and Private AI scholarship challenge nanodegree Program'. 

I study there how to protect customer personal data and still be able to train good ML models.
Here is my final project in section "Differential privacy."
In this project, I'm going to train a Differentially private model using PATE method on the MNIST dataset

### What is PATE method?
Private Aggregation of Teacher Ensembles or PATE.
In this approach, the model is not trained on sensitive data directly. 
Instead, multiple models trained on different subsets of users. These models called 'teachers.'
Then, the  'student' model trained on the noisy output of 'teachers' models.

As result 'student' model doesn't have direct access to data available to 'teachers,' but still can make accurate predictions if 'teachers' models generalize data very well.

### Points of complexity
- It is not always possible to identify people in datasets. 
       One person could write multiple handwritten digits in one set.
- Naural models rarely have the same state, even if it was trained on the same dataset.

### Task description
MNIST dataset should be separated into 'private' and 'public' parts. 'Private' datasets are available for 'teachers' only. Each 'private' dataset will contains both data(images) and targets(labels). 'Public' dataset only contains 'data'.

The goal is to answer the following questions:

- Is it possible to train 'student' without a massive loss in prediction quality?

- What is a necessary amount of noise to add to keep data private?

In [1]:
import numpy as np
import torch

In [2]:
import torchvision.datasets as datasets
from torchvision import transforms
import torch.utils.data as data_utils

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,)),
                              ])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_trainloader = data_utils.DataLoader(mnist_trainset, batch_size=64, shuffle=True)

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
mnist_testloader = data_utils.DataLoader(mnist_testset, batch_size=64, shuffle=True)

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [3]:
print(f"Mnist train size {len(mnist_trainset)}")
print(f"Mnist test size {len(mnist_testset)}")

Mnist train size 60000
Mnist test size 10000


In [4]:
# devide original dataset to an equal size subsets and return as dataloaders
def teachers_and_student_datasets(original_dataset, subset_size, parts):
    total = len(original_dataset)
    lengths = [subset_size] * parts + [total - subset_size * parts]
    print(f"Subsets lengths {lengths}")
    datasets = data_utils.dataset.random_split(original_dataset, lengths)
    return (datasets[:parts - 1], datasets[parts-1])
    
def to_dataloader(dataset):
    return data_utils.DataLoader(dataset, batch_size=64, shuffle=False)

In [5]:
from collections import namedtuple

DataInfo = namedtuple('DataInfo', 'name train test')

In [6]:
def init_teachers_data(teachers_names, teachers_trainsets, teachers_testsets):
    teachers_train_dl = [to_dataloader(d) for d in teachers_trainsets]
    teachers_test_dl = [to_dataloader(d) for d in teachers_testsets] 

    teachers_data =[DataInfo(a[0], a[1], a[2]) for a in zip(teachers_names, teachers_train_dl, teachers_test_dl)]
    return teachers_data

def init_students_data(student_name, student_trainset, student_testset):
    # train set doesn't have lables
    student_train_dl = to_dataloader([img for img, lbl in student_trainset])
    student_test_dl = to_dataloader(student_testset)

    student_data = DataInfo(student_name, student_train_dl, student_test_dl)
    return student_data

Before starting to use bigger datasets and many teachers lets test our infrastructure on toy dataset. In datasets there are only 6 'teachers'with set of 100 images each

In [11]:
teachers_names = ['El', 'Max', 'Dustin', 'Will', 'Lukas', 'Mike']
student_name = "Demogorgon"
subset_size = 100

In [12]:
N = len(teachers_names)
teachers_trainsets, student_trainset = teachers_and_student_datasets(mnist_trainset, subset_size, N + 1)
teachers_testsets, student_testset = teachers_and_student_datasets(mnist_testset, subset_size, N + 1)

teachers_data = init_teachers_data(teachers_names, teachers_trainsets, teachers_testsets)
student_data = init_students_data(student_name, student_trainset, student_testset)

Subsets lengths [100, 100, 100, 100, 100, 100, 100, 59300]
Subsets lengths [100, 100, 100, 100, 100, 100, 100, 9300]


In [13]:
print(f"{student_data}")
print(f"{next(iter(student_data.train))}")

DataInfo(name='Demogorgon', train=<torch.utils.data.dataloader.DataLoader object at 0x122272860>, test=<torch.utils.data.dataloader.DataLoader object at 0x1222882e8>)
tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
       

In [14]:
print(f"{len(teachers_data)}")
print(f"{teachers_data[2]}")
print(f"{next(iter(teachers_data[2].train))}")

6
DataInfo(name='Dustin', train=<torch.utils.data.dataloader.DataLoader object at 0x122288da0>, test=<torch.utils.data.dataloader.DataLoader object at 0x1222ad1d0>)
[tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
        

In [15]:
def prediction_accuracy(true_labels, pred_labels):
    equals = pred_labels == true_labels.view(*pred_labels.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [16]:
from torch import nn, optim
import torch.utils.data as ds
import functools

class ModelTeacher():
    INPUT_SIZE = 784
    OUTPUT_SIZE = 10
    
    def __init__(self, name, trainloader, testloader):
        super().__init__()
        self.trainloader = trainloader
        self.testloader = testloader
        self.name = name
        self.train_epochs = 5
        self._accuracy = 0
        self._model = None
        self._trained = False
        
    @property
    def model(self):
        if self._model is None:
            self._model = nn.Sequential(nn.Linear(self.INPUT_SIZE, 256),
                                        nn.ReLU(),
                                        nn.Dropout(0.2),
                                        nn.Linear(256, 64),
                                        nn.ReLU(),
                                        nn.Dropout(0.2),
                                        nn.Linear(64, self.OUTPUT_SIZE),
                                        nn.LogSoftmax(dim=1))
        return self._model
                                        
    @property
    def is_trained(self):
        return self._trained
    
    @property
    def accuracy(self):
        return self._accuracy
    
    def predict_labels(self, images):
        images = self.__flatten_input__(images)
        pred = torch.exp(self.model(images))
        # get one with highest probablity
        top_prob, top_class = pred.topk(1, dim=1)
        return top_class

    def train_model(self, verbose=True):
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.005)
        self.__opt_print__(f"{self.name} starts training", verbose)
        for e in range(self.train_epochs):
            for images, labels in self.trainloader:
                images = self.__flatten_input__(images)
                optimizer.zero_grad()
                logits = self.model(images)
                loss = criterion(logits, labels)
                loss.backward()
                optimizer.step()
            else:
                accuracy = 0
                self.model.eval()
                with torch.no_grad():
                    for images, labels in self.testloader:
                        pred_labels = self.predict_labels(images)
                        accuracy += prediction_accuracy(labels, pred_labels)
                self.model.train()
                self._accuracy = accuracy/len(self.testloader)
                self.__opt_print__(f"improved accuracy: {self._accuracy}", verbose)
        self._trained = True
        self.__opt_print__(f"{self.name} finished training", verbose)
        return self.model
    
    def __flatten_input__(self, images):
        images = images.view(images.shape[0], -1)
        assert(images.shape[1] == self.INPUT_SIZE)
        return images
    
    def __opt_print__(self, message, verbose=True):
        if verbose:
            print(message)
    

In [17]:
teachers = []
for (name, trainloader, testloader) in teachers_data:
    t = ModelTeacher(name, trainloader, testloader)
    t.train_model()
    teachers.append(t)

El starts training
improved accuracy: 0.4053819477558136
improved accuracy: 0.4409722089767456
improved accuracy: 0.4904513955116272
improved accuracy: 0.5868055820465088
improved accuracy: 0.5963541269302368
El finished training
Max starts training
improved accuracy: 0.1770833432674408
improved accuracy: 0.5381944179534912
improved accuracy: 0.4904513955116272
improved accuracy: 0.6362847089767456
improved accuracy: 0.6519097089767456
Max finished training
Dustin starts training
improved accuracy: 0.3845486044883728
improved accuracy: 0.4887152910232544
improved accuracy: 0.65625
improved accuracy: 0.6953125
improved accuracy: 0.7543402910232544
Dustin finished training
Will starts training
improved accuracy: 0.2734375
improved accuracy: 0.4105902910232544
improved accuracy: 0.5807291269302368
improved accuracy: 0.6206597089767456
improved accuracy: 0.6675347089767456
Will finished training
Lukas starts training
improved accuracy: 0.1319444477558136
improved accuracy: 0.46614581346511

In [18]:
def predict_labels_per_teacher(images, teachers):
    batch_size = images.shape[0]
    preds = np.empty([0, batch_size], dtype=int)
    for t in teachers:
        p = t.predict_labels(images).view(1, batch_size).numpy()
        preds = np.append(preds, p, axis=0)
    return preds

In [19]:
def select_best_labels(preds, use_noise=True, eps=0.25, num_labels=10):
    num_samples = preds.shape[1]
    labels = np.empty([0], dtype=int)
    for i in range(num_samples):
        counts = np.bincount(preds[:, i], minlength=num_labels)
        if use_noise:
            sensitivity = 1
            beta = sensitivity / eps
            noise = np.random.laplace(0, beta, len(counts)).astype(int)
            counts += noise
        labels = np.append(labels, np.argmax(counts))
    return labels

In [20]:
from syft.frameworks.torch.differential_privacy import pate

def perform_analysis(imageset, teachers, use_noise=True, eps=0.25, num_labels=10):
    all_preds = np.empty([len(teachers), 0], dtype=int)
    all_labels = np.empty([0], dtype=int)
    for images in imageset:
        preds = predict_labels_per_teacher(images, teachers)
        labels = select_best_labels(preds, use_noise=use_noise, eps=eps, num_labels=num_labels)
        all_preds = np.append(all_preds, preds, axis=1)
        all_labels = np.append(all_labels, labels)
    return pate.perform_analysis(teacher_preds=all_preds, indices=all_labels, noise_eps=eps, delta=1e-5) 

W0722 22:08:24.451100 4559041984 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/miniconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0722 22:08:24.465064 4559041984 deprecation_wrapper.py:119] From /miniconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [21]:
data_dep_eps, data_ind_eps = perform_analysis(student_data.train, teachers, use_noise=True, eps=0.25, num_labels=10)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps) 

add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise
add noise


In [22]:
def labeled_dataset(imageset, teachers, use_noise=True, eps=0.25, num_labels=10):
    datasets = []
    for images in imageset:
        preds = predict_labels_per_teacher(images, teachers)
        labels = select_best_labels(preds, use_noise=use_noise, eps=eps, num_labels=num_labels)
        datasets.append(data_utils.TensorDataset(images, torch.tensor(labels)))
    return data_utils.ConcatDataset(datasets)

In [23]:
import sys

def model_accuracy_range(model, dataloader, trials=15):
    min_accuracy = sys.float_info.max - 1; max_accuracy = 0
    for i in range(trials):
        accuracy = 0 
        for images, true_labels in dataloader:
            labels = model.predict_labels(images)
            accuracy += prediction_accuracy(true_labels, labels)
        accuracy = (accuracy / len(mnist_testloader)).numpy()
        min_accuracy = min(min_accuracy, accuracy)
        max_accuracy = max(max_accuracy, accuracy)
    return (min_accuracy, max_accuracy)

In [24]:
def run(teachers_names, student_name, train_subset_size, test_subset_size, use_noise=True, eps=0.25, num_labels=ModelTeacher.OUTPUT_SIZE):
    # generate data from MNIST
    N = len(teachers_names)
    teachers_trainsets, student_trainset = teachers_and_student_datasets(mnist_trainset, train_subset_size, N + 1)
    teachers_testsets, student_testset = teachers_and_student_datasets(mnist_testset, test_subset_size, N + 1)

    teachers_data = init_teachers_data(teachers_names, teachers_trainsets, teachers_testsets)
    student_data = init_students_data(student_name, student_trainset, student_testset)
    
    # train teachers models
    teachers = []
    for (name, trainloader, testloader) in teachers_data:
        t = ModelTeacher(name, trainloader, testloader)
        t.train_model(verbose=False)
        print(f"Model {name} final accuracy {t.accuracy}")
        teachers.append(t)
    
    # evaluate Epsilon
    data_dep_eps, data_ind_eps = perform_analysis(student_data.train, teachers, use_noise=use_noise, eps=eps, num_labels=num_labels)
    print("Data Independent Epsilon:", data_ind_eps)
    print("Data Dependent Epsilon:", data_dep_eps)   

    # create labeled dataset
    result_dataset = labeled_dataset(student_data.train, teachers, use_noise=use_noise, eps=eps, num_labels=num_labels)
    result_dataloader = data_utils.DataLoader(result_dataset, batch_size=64, shuffle=True)
    result_model = ModelTeacher(student_data.name, result_dataloader, student_data.test)
    result_model.train_model()
    print(f"New model accuracy {result_model.accuracy}")
    
    # get new model accuracy range
    min_accuracy, max_accuracy = model_accuracy_range(result_model, mnist_testloader, trials=15)
    print(f"New model accuracy (big dataset) range {min_accuracy}..{max_accuracy}")

In [25]:
N = 49
teachers_names = [f'office_{i}' for i in range(49)]
student_name = "Reseach office"

In [26]:
train_subset_size = 100
test_subset_size = 100
run(teachers_names, student_name, train_subset_size, test_subset_size, use_noise=True, eps=0.2)

Subsets lengths [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 55000]
Subsets lengths [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 5000]
Model office_0 final accuracy 0.6814236044883728
Model office_1 final accuracy 0.6475694179534912
Model office_2 final accuracy 0.7126736044883728
Model office_3 final accuracy 0.546875
Model office_4 final accuracy 0.6276041269302368
Model office_5 final accuracy 0.6440972089767456
Model office_6 final accuracy 0.7482638955116272
Model office_7 final accuracy 0.6710069179534912
Model office_8 final accuracy 0.7065972089767456
Model office_9 final a

In [27]:
train_subset_size = 100
test_subset_size = 100
run(teachers_names, student_name, train_subset_size, test_subset_size, use_noise=False, eps=0.2)

Subsets lengths [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 55000]
Subsets lengths [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 5000]
Model office_0 final accuracy 0.6423611044883728
Model office_1 final accuracy 0.6310763955116272
Model office_2 final accuracy 0.8446180820465088
Model office_3 final accuracy 0.6649305820465088
Model office_4 final accuracy 0.5902777910232544
Model office_5 final accuracy 0.5920138955116272
Model office_6 final accuracy 0.6553819179534912
Model office_7 final accuracy 0.7404513955116272
Model office_8 final accuracy 0.6414930820465088
Model office

In [28]:
train_subset_size = 1200
test_subset_size = 200
run(teachers_names, student_name, train_subset_size, test_subset_size, use_noise=True, eps=0.2)

Subsets lengths [1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 0]
Subsets lengths [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 0]
Model office_0 final accuracy 0.88671875
Model office_1 final accuracy 0.9140625
Model office_2 final accuracy 0.84765625
Model office_3 final accuracy 0.796875
Model office_4 final accuracy 0.8984375
Model office_5 final accuracy 0.8515625
Model office_6 final accuracy 0.9140625
Model office_7 final accuracy 0.88671875
Model office_8 final accuracy 0.85546875
Model office_9 final accuracy 0.91015625
Model 

In [29]:
train_subset_size = 1200
test_subset_size = 200
run(teachers_names, student_name, train_subset_size, test_subset_size, use_noise=False, eps=0.2)

Subsets lengths [1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 1200, 0]
Subsets lengths [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 0]
Model office_0 final accuracy 0.8828125
Model office_1 final accuracy 0.90234375
Model office_2 final accuracy 0.87890625
Model office_3 final accuracy 0.80078125
Model office_4 final accuracy 0.9453125
Model office_5 final accuracy 0.89453125
Model office_6 final accuracy 0.91015625
Model office_7 final accuracy 0.91015625
Model office_8 final accuracy 0.87890625
Model office_9 final accuracy 0.8984375
Mod

### Disclaimer

In this work was [PySyft](https://github.com/OpenMined/PySyft) was used to perform analisys. This library still in its early days. It is too early to use it to protect customer data. But in couple of months this statement should be changed